In [113]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences

In [121]:
def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def print_obj(obj, i=1, params=["agent", "interaction", "space", "time", "causal link", "physical law"]):
    print(f"{i} - ")
    for param in params:
        print(f"{param}: {obj[param]}")
    

def parse_func_resp(resp, obj_name="element", params=["name", "text"], print="True"):
    args_raw = resp.choices[0].message.tool_calls[0].function.arguments
    raws = json.loads(args_raw)[obj_name]

    obj_lst = []
    for i, obj in enumerate(raws, 1):
        if print:
            pass
            #print_obj(obj, i, params)
        obj_lst.append(obj)
    
    return obj_lst

def elements_to_string(dims, label="ELEMENTS"):
    s = ""
    for dim, elems in dims.items():
        s += f"{dim}\n{label}:\n"
        for elem in elems:
            s += elem + "\n"
        s += "\n"

    return s

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt

def ask_func(sys_msg, usr_msg, FUNC):
    resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
            tools=[FUNC], tool_choice="auto", temperature=0.5
            )
    return resp

def sample_elems(elements, per_dim, per_elem):
    total_dim = len(elements)
    ndims = min(int(per_dim * total_dim), total_dim)

    # Sample dimension indices
    dimension_indices = random.sample(range(total_dim), ndims)

    sample_elems = {}

    for di in dimension_indices:
        dimension_name = list(elements.keys())[di]
        dimension_elems = list(elements.values())[di]

        total_elems = len(dimension_elems)
        nelems = min(int(per_elem * total_elems), total_elems)

        # Sample element indices from this dimension
        element_indices = random.sample(range(total_elems), nelems)
        sample_elems[dimension_name] = [dimension_elems[ei] for ei in element_indices]

    return sample_elems

def clean_elements(elements_raw):
    elements = {}
    for element_raw in elements_raw:
        k = element_raw["name"]
        v = element_raw["text"]
        element = (k, [item.strip() for item in v.split('\n') if item.strip()])
        elements[element[0]] = element[1]
    return elements
    



load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library.sqlite"
MAX_RESULTS = 100

In [152]:
class story(BaseModel):
    agent: str
    interaction: str
    space: str
    time: str
    link: str
    law: str

    
SYS_PROMPT_ELEMENTS = """
My goal is a description of the mechanical forces driving the development of the hindgut and an understanding of how the overexpression of myosin 1c drives the inversion of LR asymmetry 
during development. I want to write a complete story of hindgut development incorporating mechanical force as central players in this story, 
then create a complete story of hindgut development under Myo1C OE, the generation of LR inversion from the role of Myo1C to the hindgut rotation. 
The following text is from a paper on hindgut morphogenesis.
Extract a list of agents, interactions, spatial compartments, developmental stages, causal chains, and physical constraints involved in hindgut morphogenesis and the effect of myosin 1c overexpression on LR inversion.
Structure the output as a story scaffold:
Agents: ...
Interactions: ...
Spaces: ...
Time: ...
Causal links: ...
Physical laws/constraints: ...
"""


SYS_PROMPT_DIMENSIONS = """
You are helping build a complete narrative of hindgut morphogenesis in Drosophila, with special attention to the role of mechanical forces - driving the rotation and elongation of the hindgut - and the overexpression of myosin 1c (Myo1C) in left-right (LR) inversion.

Your task is not to tell the story, but to define what would be required to tell a **complete and comprehensive story** for this set of story elements.

- Break down these elements into their essential **story dimensions** or **sub-components**.
- Think about biological scale (molecular, cellular, tissue), space, time, dynamics, agents, interactions, functions, and unknowns.
- Consider how these elements might relate to:
  - other parts of the hindgut
  - the mechanical forces shaping the tissue
  - the overexpression of Myo1C and its effects on LR inversion

Output a structured list of **story dimensions** that should be explored in order to fully develop these elements. If helpful, include example questions or variables within each dimension.

Format:

**1. Dimension Name**  
- Description of what this dimension involves.  
- Example questions or aspects to explore.

(Repeat as needed for all relevant dimensions)
"""


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")


SYS_PROMPT_STORYTELLING = """
You are constructing a complete and mechanistically rich **narrative of Drosophila hindgut morphogenesis**, spanning from **stage 11 to stage 15**, with a particular focus on the role of **mechanical forces** in driving **left-right (LR) asymmetric rotation** and **anterior-posterior elongation** of the gut.

This narrative must:
- Progress through **a clear beginning, middle, and end**, anchored to specific developmental stages.
- Integrate biological elements (agents, interactions, causal links, physical laws, etc.) into a **continuous developmental story**.
- Be organized by **stages of development**, but deeply informed by **conceptual dimensions** like spatial organization, tissue morphogenesis, molecular interactions, temporal regulation, and unknowns.

---

### 🔬 BIOLOGICAL ELEMENTS TO INTEGRATE
These are grouped by type and must be woven into the narrative. Explain how they emerge, interact, or evolve across time, and how they contribute to hindgut development and LR asymmetry.

{elements_text}

---

### 🧭 DIMENSIONS OF STORY STRUCTURE
Use the following conceptual dimensions to guide the exploration of how the elements interact and function over time. Within each phase (beginning, middle, end), consider these dimensions to reveal biological meaning and causal relationships.

{dimensions_text}

---

### 📘 INSTRUCTIONS FOR STORYTELLING

- Begin at **stage 11**, describing the initial state of the hindgut and its relevant cellular, molecular, and tissue-level configurations.
- In the **middle**, describe the transformation from stage 11 to stage 14: how elements and dimensions evolve, interact, and drive morphogenetic change.
- Conclude at **stage 15**, describing the final morphology, internal organization, and functional integration of the hindgut.

At each stage:
- Use the **dimensions** as lenses to explore how the elements operate and change.
- Show how mechanical forces, molecular regulators, spatial constraints, and timing orchestrate morphogenesis.
- Ensure developmental sequencing is causally and spatially coherent.

The story should flow like a scientific narrative that unfolds through time, with mechanistic insight and biological precision.

---

### 🔚 ENDING THE STORY

Conclude with:
- A synthesis of how the initial configuration transformed into the final morphology.
- Key causal mechanisms and checkpoints.

"""

SYS_PROMPT_STORYTELLING2 = """
You are rewriting a complete developmental narrative of **Drosophila hindgut morphogenesis from stage 11 to stage 15**, under the condition that **Myosin 1C (Myo1C) is overexpressed across the brachyenteron domain**, leading to **inversion of left-right (LR) asymmetry**.

The original story below describes normal hindgut development through molecular, cellular, tissue, and mechanical dimensions.

---

### 📖 Original Wild-Type Narrative:
{wt_story}

---

### 🎯 New Objective: Mutation-Driven Story of LR Inversion

You must now **retell the full developmental story** from stage 11 to stage 15, describing how **Myo1C overexpression disrupts, rewires, or distorts the normal developmental trajectory**.

Structure the story as a **temporal narrative** with:

---

### I. **BEGINNING (Stage 11)**  
Describe the state of the hindgut at stage 11 *under Myo1C OE*. Which molecular players, spatial structures, and physical forces are already altered? What remains similar to wild-type?

---

### II. **MIDDLE (Transition through Stage 12–14)**  
Trace the dynamic changes in morphology, mechanical force propagation, cellular behaviors, gene expression, and spatial orientation.  
- What **new causal chains** emerge?
- How are existing **feedbacks, constraints, or symmetry cues disrupted**?
- Which **interactions or dimensions break down or invert**?
- Which **structures and dynamics remain the unaffected**?

---

### III. **END (Stage 15 Outcome)**  
What is the **final anatomical and functional state** of the hindgut under Myo1C OE?  
- Describe the **phenotypic result**, such as LR inversion or malformed rotation.
- Explain how the cascade of altered elements leads to this specific outcome.

---

### 📘 Use the following conceptual dimensions as lenses throughout the narrative:
{dimensions_text}

These should guide your mechanistic reasoning within each time window. Explain how each dimension is impacted by the overexpression of Myo1C and contributes to the inverted outcome.

---

### 🔬 Reintegrate Relevant Elements:
{elements_text}

Explicitly connect these elements to the evolving story. Focus on **mechanical, molecular, spatial, and causal specificity**.

---

### ✅ Guidelines:
- Maintain a **scientific tone** grounded in known biology and plausible hypotheses.
- Emphasize **mechanistic detail**: how does Myo1C overexpression perturb cytoskeletal tension, spatial polarity, adhesion, or developmental timing?
- If Myo1C dominates certain molecular interactions or creates feedback loops, show how these propagate to higher levels.
- Be consistent with developmental stage timing and spatial logic.
- Where evidence is incomplete, **propose testable hypotheses**.

---

### 🔚 Conclusion:
Synthesize the full inversion narrative:
- What are the **key causal disruptions**?
- How does Myo1C act as the initiating factor?

"""

SYSTEM_PROMPT_REVIEW = """"
You are reviewing a developmental biology narrative of Drosophila hindgut morphogenesis. Your task is to:
- Extract all statements that are **plausible but hypothetical or inferred**.
- Clearly distinguish these two categories and highlight any assumptions.

Format:
- Hypotheses: [...]
- Assumptions: [...]
"""

SYSTEM_PROMPT_ALTERNATIVE = """
You are reviewing a developmental biology narrative about Drosophila hindgut morphogenesis and the role of Myo1C in LR asymmetry inversion. The following hypotheses are currently assumed:

{review}

Your task is to generate **3-5 alternative, mechanistically distinct hypotheses**, each operating at different biological scales or representing different hypothesis types.

For each alternative:
- Provide a short title.
- Indicate the **biological scale** (molecular, cellular, tissue, system).
- Classify the **hypothesis type** (causal, structural, regulatory, compensatory, etc.).
- Describe the **mechanism**.
- Explain how it **differs** from the original hypothesis.
- Include one **prediction** (what would you observe if this were true vs. not?).

Only return the list of hypotheses with these annotations. Format clearly in markdown.
"""

SYSTEM_PROMPT_MUTATE = """
You are a scientific storyteller and developmental biology expert. Your task is to rewrite a comprehensive narrative of Drosophila hindgut development using a set of **alternative hypotheses** that describe different plausible mechanisms of left-right asymmetry and tissue morphogenesis.

Use the original story's structure, but update the mechanistic explanations, causal flows, and biological logic so that the **entire narrative reflects a synthesis of the alternative hypotheses**.

Where mechanisms differ or conflict, propose a unified framework or explain how multiple mechanisms may interact or act in parallel.

The original story was constructed using the following story elements and narrative dimensions. Preserve these structures unless directly contradicted by the alternative hypotheses.

If needed, modify specific elements or dimensions to reflect the new mechanisms and causal flows introduced by the alternative hypotheses.

Goal: Generate one **alternative full story** that reflects and integrates these alternative hypotheses into a **coherent and biologically plausible morphogenetic narrative**.
"""

USER_PROMPT_MUTATE = """
ORIGINAL STORY:
{wt_narrative}

ORIGINAL STORY ELEMENTS:
{elements_text}

ORIGINAL STORY DIMENSIONS:
{dimensions_text}

ALTERNATIVE HYPOTHESES TO INTEGRATE:
{alt_hyp}

Please write a single comprehensive story of Drosophila hindgut morphogenesis that reflects and integrates these alternative hypotheses while aligning with or updating the original elements and dimensions as needed.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}

In [10]:
KEYWORDS = ["active tension network", "epithelial"]  # list of words
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# papers = {}
for r in search_results:
    title = r['title']
    text = r['text']
    print(title, len(text))

Active Tension Network Model in Epithelial Tissues 32334
The Geometric Basis of Epithelial Convergent Extension 188479
Generating Active T1 Transitions through Mechanochemical Feedback 117183


In [9]:
KEYWORDS = ["active tension network", "epithelial"]  # list of words
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# papers = {}
for r in search_results:
    title = r['title']
    text = r['text']
    papers[title] = text


rem = [None, "Fundamental Neuroscience (3rd edition)","Harrison's Principles of Internal Medicine 15th Edition", "Developmental Biology, 7th Edition"]
for r in rem:
    if r in papers:
        del papers[r]


for title in list(papers.keys()):
    print(title)

The Drosophila actin nucleator DAAM is essential for left-right asymmetry
Myosin1D is an evolutionarily conserved regulator of animal left–right asymmetry
An unconventional myosin in Drosophila reverses the default handedness in visceral organs
Chiral cell sliding drives left-right asymmetric organ twisting
Cell Chirality Drives Left-Right Asymmetric Morphogenesis
Chirality in Planar Cell Shape Contributes to Left-Right Asymmetric Epithelial Morphogenesis
Vertebrate myosin 1d regulates left–right organizer morphogenesis and laterality
Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila
Understanding Laterality Disorders and the Left-Right Organizer: Insights from Zebrafish
Generating Active T1 Transitions through Mechanochemical Feedback
Left–right Myosin-Is, Myosin1C, and Myosin1D exhibit distinct single molecule behaviors on the plasma membrane of Drosophila macrophages
It Takes Guts: The Drosophila Hindgut a

In [131]:
for n in range(1):
    print(n)

0


In [136]:
stories = {}

for n in range(1): # range(len(papers)):
    title, text = list(papers.keys())[n], list(papers.values())[n]
    excerpt = sample_text(text, 10000)

    sys_msg = SYS_PROMPT_ELEMENTS
    user_msg = f"TEXT EXCERPT:\n{excerpt}"
    elements_resp = ask(sys_msg, user_msg)

    sys_msg = SYS_PROMPT_PARSE_ELEMENTS
    usr_msg = f"STORY FRAMEWORK:\n{elements_resp}"
    elements_func = ask_func(sys_msg, usr_msg, ELEM_FUNC)

    elements_raw = parse_func_resp(elements_func, "element", ["name", "text"], "False")
    elements = clean_elements(elements_raw)

    per_dim, per_elem = 1, 1   # Number of elements per dimension to sample
    elements_sample = sample_elems(elements, per_dim, per_elem)
    elements_text = elements_to_string(elements_sample, "ELEMENTS")

    # GET DIMENSIONS
    sys_msg = SYS_PROMPT_DIMENSIONS
    usr_msg = f"Here are the story elements:\n\n{elements_text}"
    dimensions_resp = ask(sys_msg, usr_msg)

    sys_msg = SYS_PROMPT_PARSE_ELEMENTS
    usr_msg = f"STORY DIMENSIONS:\n{dimensions_resp}"
    dimensions_func = ask_func(sys_msg, usr_msg, ELEM_FUNC)

    dimensions_raw = parse_func_resp(dimensions_func, "element", ["name", "text"], "False")
    dimensions = clean_elements(dimensions_raw)

    # Configuration
    per_dim, per_elem = 1, 1 
    dimensions_sample = sample_elems(dimensions, per_dim, per_elem)
    dimensions_text = elements_to_string(dimensions_sample, "DIMENSIONS")

    sys_msg = SYS_PROMPT_STORYTELLING.format(elements_text=elements_text, dimensions_text=dimensions_text)
    usr_msg = "Please generate the complete, structured narrative based on the elements and dimensions provided."
    wt_resp = ask(sys_msg, usr_msg)

    sys_msg = SYS_PROMPT_STORYTELLING2.format(wt_story=wt_resp, elements_text=elements_text, dimensions_text=dimensions_text)
    usr_msg = "Please rewrite the original story to reflect the overexpression of Myosin 1C (Myo1C) across the brachyenteron domain. Show how this perturbation leads to inversion of left-right asymmetry in the Drosophila hindgut by altering the behavior of affected molecular, cellular, and tissue-level elements. Maintain the original story structure and focus on causal, mechanistic explanations."
    myo_resp = ask(sys_msg, usr_msg)

    stories[title] = {
        "elements": elements_text,
        "dimensions": dimensions_text,
        "wt_story": wt_resp,
        "myo_story": myo_resp
    }


    print(title, "\n\n")
    print(wt_resp, "\n\n")
    print(myo_resp, "\n\n\n")

The Drosophila actin nucleator DAAM is essential for left-right asymmetry 


## Beginning: Stage 11

### **Stage 11: Initial State of the Hindgut**

In the developmental stage 11 of Drosophila, the hindgut represents a nascent structure poised for substantial morphogenetic transformation. At this early stage, cells within the hindgut are characterized by a primitive arrangement, displaying basic adherens junctions critical for future mechanical integrity and cellular communication. The molecular landscape comprises key agents such as DE-cadherin, N-Cadherin, and β-catenin, which form the backbone of cellular adhesion sites, governing interactions crucial for subsequent morphogenesis.

**Molecular Agents and Cellular Interactions**
- **DE-cadherin and β-catenin** establish adherens junctions, facilitating cell-cell adhesion that maintains tissue cohesion. β-catenin acts as a critical linker between cadherins and the actin cytoskeleton.
- **DAAM (Disheveled Associated Activator of Morpho

In [153]:
story = list(stories.values())[0]
wt_narrative = story["wt_story"]
inv_narrative = story["myo_story"]
#elements_text = story["elements"]
#dimensions_text = story["dimensions"]
sys_msg = SYSTEM_PROMPT_REVIEW
usr_msg = f"Here is the narrative of hindgut morphogenesis:\n\n{wt_narrative}"
for _ in range(1):
    review = ask(sys_msg, usr_msg)
    sys_prompt_hypotheses = SYSTEM_PROMPT_ALTERNATIVE.format(review=review)
    usr_prompt_hypotheses = f"Here is the original narrative of hindgut morphogenesis:\n\n{wt_narrative}"
    alt_hyp = ask(sys_prompt_hypotheses, usr_prompt_hypotheses)
    print(f"REVIEW:\n{review}\n\n")
    print(f"ALTERNATIVE HYPOTHESES:\n{alt_hyp}\n\n\n")

REVIEW:
- Hypotheses: 
  - [The formation of the chiral cytoskeleton by DAAM-Myo1D complexes is crucial at adherens junctions for the next phase of cell alignment and tissue rotation.]
  - [The interaction of DAAM with Profilin (Chic) begins to nucleate actin filaments, providing a framework that will be fine-tuned towards LR asymmetry.]
  - [Gentle rotation directed by microtubules and actin filaments manifests subtle LR differentiation.]
  - [Myo1C's overexpression leads to sinistral phenotypes by modulating DAAM-induced chiral F-actin networks, unveiling how genetic variants alter developmental choreography.]
  - [The anterior-posterior axis expansion is propelled by forces resultant from strategic cellular reorientation and ECM reformation.]
  - [Adequate levels of Myo1C overexpression, when left unchecked, have shown potential for an inverted left-right orientation due to their role in remodelling chiral structures.]

- Assumptions:
  - [The molecular landscape at stage 11 with ke

In [142]:


sys_msg = SYSTEM_PROMPT_MUTATE
usr_msg = USER_PROMPT_MUTATE.format(wt_narrative=wt_narrative, elements_text=elements_text, dimensions_text=dimensions_text, alt_hyp=alt_hyp)
wt_story2 = ask(sys_msg, usr_msg)
print(wt_story2)

# Alternative Story: Comprehensive Morphogenetic Narrative of Drosophila Hindgut Development

## Beginning: Stage 11

### **Stage 11: Initial State of the Hindgut**

In the nascent stages of Drosophila development, the hindgut emerges as an essential structure set upon a path of complex morphogenetic transformation. At stage 11, the hindgut is marked by rudimentary cellular arrangements characterized by primitive adherens junctions, crucial for upholding mechanical integrity and facilitating cellular communication. These nascent structural features are coordinated by a molecular landscape rich with components such as DE-cadherin, N-Cadherin, and β-catenin, which form the primary scaffold of cellular adhesion, paramount for ensuing morphogenesis.

**Molecular Agents and Cellular Interactions**
- **DE-cadherin and β-catenin** solidify adherens junctions, providing cell-cell adhesion necessary for tissue coherence. β-catenin also links cadherins to the actin cytoskeleton, which works alon

In [145]:
SYSTEM_PROMPT_INVERSION2 = """You are a scientific narrator and developmental systems theorist. Your task is to revise a comprehensive narrative of Drosophila hindgut morphogenesis that was previously constructed based on a set of mechanistic hypotheses (e.g., microtubule-based asymmetry, ECM remodeling, membrane lipid patterning, environmental sensitivity).

This new version should explore how **overexpression of Myosin 1C (Myo1C)** perturbs or rewires the developmental trajectory to result in an inversion of LR asymmetry during the rotation of the gut in stage 14.

Maintain the **structure, flow, and assumptions** of the original story derived from alternative hypotheses. Your goal is not to revert to canonical stories, but to **mutate this alternative framework** in light of the new perturbation.

### Objective:
- Mutate the story using the same **stage-wise sections**, **elements**, and **dimensions** of the original.
- Show which components (e.g., microtubules, lipid domains, feedback loops) are affected, destabilized, or resilient.
- Introduce causal mechanisms by which **Myo1C overexpression** leads to **inversion of left-right asymmetry**, altered mechanics, or disrupted feedback systems.
- Be specific about molecular, cellular, and tissue-level consequences of the perturbation.
- Highlight whether Myo1C acts upstream, downstream, or orthogonal to the alternative mechanisms.
- If unknowns exist, hypothesize plausible integrative explanations.

### Guidelines:
- Do not change the story structure (Beginning, Middle, End).
- Preserve references to the alternative hypotheses, unless they are directly invalidated or rewritten by the Myo1C effect.
- Be mechanistic, not vague. Focus on **how** Myo1C’s overexpression alters physical and molecular interactions.
- Maintain clarity and flow. Preserve biological realism and grounded speculation.

At the end, optionally summarize how each alternative hypothesis responds or adapts under Myo1C overexpression."""

USER_PROMPT_INVERSION = f"""
Here is the alternative hypothesis-based narrative of Drosophila hindgut morphogenesis:

{wt_story2}

Now, rewrite this story under the condition that:

**Myo1C is overexpressed across the brachyenteron domain.**

This overexpression perturbs the system and results in **inversion of left-right asymmetry**.

Use the existing alternative hypotheses and mechanistic layers (e.g., microtubule mechanics, lipid patterning, ECM remodeling, compensatory Myo1 behavior, etc.) to **reweave the story**, showing how Myo1C overexpression cascades through the system.

STORY ELEMENTS:
{elements_text}

STORY DIMENSIONS:
{dimensions_text}

ALTERNATIVE HYPOTHESES:
{alt_hyp}

Keep the original structure. Rewrite all sections from Stage 11 through Stage 15.
"""


sys_msg = SYSTEM_PROMPT_INVERSION2
usr_msg = USER_PROMPT_INVERSION
myo_resp2 = ask(sys_msg, usr_msg)
print(myo_resp2)

# Rewritten Story: Myo1C Overexpression's Impact on Hindgut Morphogenesis

## Beginning: Stage 11

### **Stage 11: Initial State of the Hindgut**

At stage 11, the foundational structure of the Drosophila hindgut is established, now experiencing a profound perturbation due to Myo1C overexpression. The hindgut's rudimentary cellular arrangements, typically marked by nascent adherens junctions and molecular cohesion through DE-cadherin, N-Cadherin, and β-catenin, begin showing alterations in mechanical integrity and cellular communication pathways.

**Molecular Agents and Cellular Interactions**
- **DE-cadherin and β-catenin** establish the baseline of cellular adhesion, yet their interplay with the actin cytoskeleton is now influenced by excessive Myo1C presence. Myo1C's interaction with actin potentially creates aberrant force vectors, altering early-stage mechanical constraints.
- **Microtubules**, typically driving mechanical torsion and orientation, become participants in a more com

### New Narrative: Myosin 1C Overexpression and LR Inversion in Drosophila Hindgut Development

## I. **BEGINNING (Stage 11): Initial State of the Hindgut with Myo1C Overexpression**

In the early phases of Drosophila embryogenesis, the hindgut is poised for morphogenetic progression. At stage 11, with the overexpression of Myosin 1C (Myo1C) across the brachyenteron domain, fundamental changes begin to unfold in the cellular landscape. While similar molecular adherens junction components like DE-cadherin, N-Cadherin, and β-catenin anchor the tissue hook, the mechanical interplay is notably altered.

**Molecular Agents and Cellular Interactions**:
- **Myo1C overexpression** accentuates interactions with F-actin, interrupting normal microtubule dynamics and affecting cell polarity and adhesion setups governed by DE-cadherin and β-catenin.
- **Microtubule arrangements** become skewed, influenced by Myo1C activity, disrupting the symmetry cues that typically support preliminary LR orientat

In [156]:
SYSTEM_PROMPT_ZOOM = """
You are a scientific storyteller constructing a detailed narrative of Drosophila hindgut morphogenesis.

Your task is: to identify **elements or phrases** in the story that encode **assumptions or mechanistic decisions** and determine the *dimensions of storytelling* necessary to fully and precisely narrate this element’s role in development.

An exampe of such dimensions are, but are not limited to:
- **Gene expression**: When, where, and how strongly the genes/proteins are expressed.
- **Localization**: Subcellular or tissue localization patterns.
- **Biochemical interactions**: Binding partners, interaction dynamics, regulatory modifications.
- **Morphological context**: Structural shape, pattern, or form at different scales.
- **Temporal dynamics**: Changes across developmental timepoints.
- **Regulatory control**: Upstream regulators, feedback loops, pathway membership.
- **Functional consequences**: What happens when the element is altered or missing.
- **Scale of action**: Molecular → cellular → tissue → system-level integration.
- **Mechanotransduction**: If applicable, how mechanical forces involve this element.
- **Comparative relevance**: Variants across conditions, mutants, or species.

For each story element:
1. Quote the original phrase.
2. List 8–12 dimensions of storytelling necessary to tell the complete mechanistic story of this element in the context of Drosophila hindgut development.

Be precise, mechanistic, and narratively oriented. Avoid vague or redundant categories.
"""

USER_PROMPT_ZOOM = """Here is the story:

{story_text}

Extract and annotate dimensions of variation as described."""

sys_msg = SYSTEM_PROMPT_ZOOM
usr_msg = USER_PROMPT_ZOOM.format(story_text=wt_narrative)
zoom = ask(sys_msg, usr_msg)
print(zoom)





Here are the annotated elements and their storytelling dimensions from the Drosophila hindgut morphogenesis narrative:

### Element: "DE-cadherin and β-catenin establish adherens junctions"

1. **Gene expression**: Levels of DE-cadherin and β-catenin transcripts and proteins at Stage 11.
2. **Localization**: Subcellular localization, particularly the positioning at adherens junctions.
3. **Biochemical interactions**: Specific binding dynamics between DE-cadherin, β-catenin, and the actin cytoskeleton.
4. **Morphological context**: The structural role of adherens junctions in nascent hindgut cells.
5. **Temporal dynamics**: Timing of junction formation and its evolution through subsequent stages.
6. **Regulatory control**: Upstream signals implicated in the initiation and maintenance of DE-cadherin and β-catenin function.
7. **Functional consequences**: Effects on cell adhesion and tissue integrity when these junctions are disrupted.
8. **Scale of action**: Role from subcellular level (

In [157]:
SYSTEM_PROMPT_NARRATIVE2 = """
You are a developmental biology narrator specializing in morphogenetic storytelling.

Your task is to write a coherent, multi-stage narrative of Drosophila hindgut development using a set of annotated biological elements. Each element has a collection of "dimensions of storytelling" that represent aspects necessary for a complete, high-resolution mechanistic story.

Your job:
1. Incorporate these elements into a staged developmental timeline (e.g., Stage 11, Stage 12–14, Stage 15).
2. For each stage, weave a story that:
   - Uses the **dimensions of storytelling** to bring each element to life.
   - Connects elements causally and temporally across molecular, cellular, and tissue levels.
   - Embeds mechanistic detail without becoming a list—focus on flow, causality, and developmental logic.
3. Include emerging phenomena such as rotation, inversion, and asymmetry as outcomes of the described elements.
4. Maintain clarity, continuity, and scientific plausibility.

Each paragraph should correspond to a developmental stage and integrate relevant elements.

Be creative, integrative, and narratively precise.
"""

USER_PROMPT_NARRATIVE2 = """
Please write a new narrative about Drosophila hindgut morphogenesis using the following annotated elements and their storytelling dimensions:

{zoom}

Structure the narrative in three parts:
- **Beginning**: Early events around Stage 11
- **Middle**: Rotation and elongation during Stages 12–14
- **End**: Morphological outcomes by Stage 15

Integrate the storytelling dimensions into each part. Show how the expression patterns, localization, molecular interactions, mechanical forces, and regulatory systems create a causally coherent story across time and scale.

The final story should be mechanistic, precise, and narratively complete.
"""

sys_msg = SYSTEM_PROMPT_NARRATIVE2
usr_msg = USER_PROMPT_NARRATIVE2.format(zoom=zoom)
wt_narrative2 = ask(sys_msg, usr_msg)
print(wt_narrative2)

## Beginning: Stage 11 - Laying the Foundation

In the early dawn of Drosophila hindgut development at Stage 11, the foundation for a complex structure begins with intricate molecular choreography. Here, DE-cadherin and β-catenin emerge as the architects, setting the stage for cellular cohesion through the establishment of adherens junctions. Elevated gene expression of DE-cadherin and β-catenin ensures these molecules are adequately supplied, strategically localizing at cell-cell junctions where they bind to the actin cytoskeleton. This binding is not a mere static interaction but a dynamic dance—a molecular handshake—critical for maintaining structural integrity in nascent hindgut cells. These adherens junctions represent more than just molecular entities; they are the fabric weaving individual cells into a cohesive tissue at a subcellular scale, responding to upstream signals that continue to regulate their assembly and function. Meanwhile, DAAM takes its post as an actin nucleator,

In [158]:
print(elements_text)
print(dimensions_text)
print(wt_narrative)

Story Scaffold for Hindgut Morphogenesis and Myosin 1c Overexpression Impact on LR Inversion
ELEMENTS:

Agents
ELEMENTS:
6. DE-cadherin and β-catenin
10. Formins and F-actin
5. N-Cadherin
2. Myo1D and Myo1C (non-muscle myosin 1 isoforms)
3. Profilin (Chic)
1. DAAM (Disheveled Associated Activator of Morphogenesis)
7. PS2 integrin ligand (Tenectin)
4. Tec29
9. Actin, microtubules
8. Pitx2 and Wnt (in vertebrate context)

Interactions
ELEMENTS:
5. DAAM and Myo1 interact with cadherins controlling cell adhesion processes.
1. DAAM interacts with Profilin (Chic) for actin nucleation.
2. Myo1D and Myo1C interact with F-actin for inducing chirality.
4. N-Cadherin influences ECM for proper gut looping.
3. DAAM-Myo1D complex facilitates chiral cytoskeleton formation at adherens junctions.

Causal links
ELEMENTS:
3. Overexpression of Myo1C influences sinistral (anti-clockwise) phenotypes, altering normal LR asymmetry.
1. DAAM’s role as an actin nucleator is essential for laterality and F-actin n